In [1]:
import numpy as np
import pandas as pd
import os
import win32com.client as win32
import time
import gc
layouts = np.load("M2_data_F8_layouts.npy", allow_pickle=True)
results = np.load("M2_data_F8_results.npy", allow_pickle=True)
positions = np.load("M2_data_F8_positions.npy", allow_pickle=True)
for layout,position in zip(layouts, positions):
    if "-" in layout:
        expected_length = len(layout)-2
    else:
        expected_length = len(layout)-1
    if len(position) != expected_length:
        print(f"Layout: {layout}, Position: {position}, Expected Length: {expected_length}, Actual Length: {len(position)}")
        break
print(len(layouts), len(positions), len(results))

4267 4267 4267


In [2]:
class Simulation:
    AspenSimulation = win32.gencache.EnsureDispatch("Apwn.Document")

    def __init__(self, AspenFileName, WorkingDirectoryPath, VISIBILITY=False):
        os.chdir(WorkingDirectoryPath)
        print(f"Working Directory: {os.getcwd()}")
        print(f"Aspen File: {AspenFileName}")
        self.AspenSimulation.InitFromFile2(os.path.abspath(AspenFileName))
        self.AspenSimulation.Visible = VISIBILITY
        self.AspenSimulation.SuppressDialogs = True

    def CloseAspen(self):
        AspenFileName = self.Give_AspenDocumentName()
        self.AspenSimulation.Close(os.path.abspath(AspenFileName))

    def Give_AspenDocumentName(self):
        return self.AspenSimulation.FullName

    @property
    def BLK(self):
        return self.AspenSimulation.Tree.Elements("Data").Elements("Blocks")

    @property
    def STRM(self):
        return self.AspenSimulation.Tree.Elements("Data").Elements("Streams")

    def EngineRun(self):
        self.AspenSimulation.Run2()

    def EngineStop(self):
        self.AspenSimulation.Stop()

    def EngineReinit(self):
        self.AspenSimulation.Reinit()

    def Convergence(self):
        converged = (
            self.AspenSimulation.Tree.Elements("Data")
            .Elements("Results Summary")
            .Elements("Run-Status")
            .Elements("Output")
            .Elements("PER_ERROR")
            .Value
        )
        return converged == 0

    def StreamConnect(self, Blockname, Streamname, Portname):
        self.BLK.Elements(Blockname).Elements("Ports").Elements(Portname).Elements.Add(
            Streamname
        )

    def StreamDisconnect(self, Blockname, Streamname, Portname):
        self.BLK.Elements(Blockname).Elements("Ports").Elements(
            Portname
        ).Elements.Remove(Streamname)

    def Reinitialize(self):
        self.STRM.RemoveAll()
        self.BLK.RemoveAll()
        self.AspenSimulation.Reinit()

class Stream(Simulation):
    def __init__(self, name, inlet=False):
        self.name = name.upper()
        self.inlet = inlet

        self.StreamPlace()

        if self.inlet:
            self.inlet_stream()

    def StreamPlace(self):
        compositstring = self.name + "!" + "MATERIAL"
        self.STRM.Elements.Add(compositstring)

    def StreamDelete(self):
        self.STRM.Elements.Remove(self.name)

    def inlet_stream(self):
        T = self.inlet[0]
        P = self.inlet[1]
        comp = self.inlet[2]

        self.STRM.Elements(self.name).Elements("Input").Elements("TEMP").Elements(
            "MIXED"
        ).Value = T
        self.STRM.Elements(self.name).Elements("Input").Elements("PRES").Elements(
            "MIXED"
        ).Value = P

        for chemical in comp:
            self.STRM.Elements(self.name).Elements("Input").Elements("FLOW").Elements(
                "MIXED"
            ).Elements(chemical).Value = comp[chemical]

    def get_temp(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("TEMP_OUT")
            .Elements("MIXED")
            .Value
        )

    def get_press(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("PRES_OUT")
            .Elements("MIXED")
            .Value
        )

    def get_molar_flow(self, compound):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MOLEFLOW")
            .Elements("MIXED")
            .Elements(compound)
            .Value
        )

    def get_total_molar_flow(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MOLEFLMX")
            .Elements("MIXED")
            .Value
        )

    def get_mass_flow(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MASSFLMX")
            .Elements("MIXED")
            .Value
        )
    def stream_specs(self):
        return (self.get_temp(), self.get_press(), self.get_mass_flow())
    def get_vapor_fraction(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("STR_MAIN")
            .Elements("VFRAC")
            .Elements("MIXED")
            .Value
        )

class Block(Simulation):
    def __init__(self, name, uo):
        self.name = name.upper()
        self.uo = uo

    def BlockCreate(self):
        compositestring = self.name + "!" + self.uo
        self.BLK.Elements.Add(compositestring)

    def BlockDelete(self):
        self.BLK.Elements.Remove(self.name)

In [3]:

# -------------------------------------------------- UNIT OPERATIONS ------------------------------------------------


class Mixer(Block):
    def __init__(self, name, inlet_stream1,inlet_stream2=None):
        super().__init__(name, "Mixer")
        self.name = name
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.BlockCreate()

    def mix(self,mixer_count):
        if mixer_count == 1:
            self.StreamConnect(self.name, self.inlet_stream1.name, "F(IN)")
            s = None
        else:
            self.StreamConnect(self.name, self.inlet_stream2.name, "F(IN)")
            self.StreamConnect(self.name, self.inlet_stream1.name, "F(IN)")
            s = Stream(f"{self.name}OUT")
            self.StreamConnect(self.name, s.name, "P(OUT)")
            
        return s


class Splitter(Block):
    def __init__(self, name, sr, inlet_stream):
        super().__init__(name, "FSplit")

        self.name = name
        self.sr = sr
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def split(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        s1 = Stream(f"{self.name}S1OUT")
        s2 = Stream(f"{self.name}S2OUT")

        self.StreamConnect(self.name, s1.name, "P(OUT)")
        self.StreamConnect(self.name, s2.name, "P(OUT)")

        self.BLK.Elements(self.name).Elements("Input").Elements("FRAC").Elements(
            s1.name
        ).Value = self.sr
        return s1, s2


class Heater(Block):
    def __init__(self, name, Temp, Pres,inlet_stream):
        super().__init__(name, "Heater")

        self.name = name
        self.Temp = Temp
        self.Pres = Pres
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def heat(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        self.BLK.Elements(self.name).Elements("Input").Elements("SPEC_OPT").Value = "TP"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TEMP"
        ).Value = self.Temp
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES"
        ).Value = self.Pres

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("QCALC").Value)
        return q


class HeatExchanger(Block):
    def __init__(self, name, DT, inlet_stream1, inlet_stream2=None):
        super().__init__(name, "HeatX")

        self.name = name
        self.DT = DT
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.hotside_pres = None
        self.coldside_pres = None
        self.BlockCreate()

    def heat(self,hex_count,pres):
        if hex_count == 1:
            self.StreamConnect(self.name, self.inlet_stream1.name, "H(IN)")
            s = Stream(f"{self.name}OUT1")
            self.StreamConnect(self.name, s.name, "H(OUT)")
            self.BLK.Elements(self.name).Elements("Input").Elements(
                "SPEC"
            ).Value = "DELT-HOT"
            self.BLK.Elements(self.name).Elements("Input").Elements(
                "VALUE"
            ).Value = self.DT
            self.hotside_pres = pres
            self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_HOT").Value = self.hotside_pres
            self.outlet1 = s
        else:
            self.StreamConnect(self.name, self.inlet_stream2.name, "C(IN)")
            s = Stream(f"{self.name}OUT2")
            self.StreamConnect(self.name, s.name, "C(OUT)")
            self.coldside_pres = pres
            self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_COLD").Value = self.coldside_pres
            self.outlet2 = s
        return s
    def switch_streams(self):
        # Switch the inlet streams for the heat exchanger
        self.StreamDisconnect(self.name, self.inlet_stream1.name, "H(IN)")
        self.StreamDisconnect(self.name, self.inlet_stream2.name, "C(IN)")
        self.StreamConnect(self.name, self.inlet_stream2.name, "H(IN)")
        self.StreamConnect(self.name, self.inlet_stream1.name, "C(IN)")
        new_coldside_pres = self.hotside_pres
        new_hotside_pres = self.coldside_pres        

        # Update the outlet streams accordingly
        self.StreamDisconnect(self.name, self.outlet1.name, "H(OUT)")
        self.StreamDisconnect(self.name, self.outlet2.name, "C(OUT)")
        self.StreamConnect(self.name, self.outlet2.name, "H(OUT)")
        self.StreamConnect(self.name, self.outlet1.name, "C(OUT)")
        self.hotside_pres = new_hotside_pres
        self.coldside_pres = new_coldside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_HOT"
        ).Value = self.hotside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_COLD"
        ).Value = self.coldside_pres

    def undo_switch(self):
        # Switch the inlet streams back to their original state
        self.StreamDisconnect(self.name, self.inlet_stream2.name, "H(IN)")
        self.StreamDisconnect(self.name,self.inlet_stream1.name, "C(IN)")
        self.StreamConnect(self.name, self.inlet_stream1.name, "H(IN)")
        self.StreamConnect(self.name, self.inlet_stream2.name, "C(IN)")
        new_coldside_pres = self.hotside_pres
        new_hotside_pres = self.coldside_pres
        
        # Update the outlet streams accordingly
        self.StreamDisconnect(self.outlet2.name, "H(OUT)")
        self.StreamDisconnect(self.outlet1.name, "C(OUT)")
        self.StreamConnect(self.outlet1.name, "H(OUT)")
        self.StreamConnect(self.outlet2.name, "C(OUT)")
        self.hotside_pres = new_hotside_pres
        self.coldside_pres = new_coldside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_HOT"
        ).Value = self.hotside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_COLD"
        ).Value = self.coldside_pres

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("HX_DUTY").Value)
        return q


class Cooler(Block):
    def __init__(self, name, Temp,Pres, inlet_stream):
        super().__init__(name, "Heater")

        self.name = name
        self.Temp = Temp
        self.Pres = Pres
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def cool(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        self.BLK.Elements(self.name).Elements("Input").Elements("SPEC_OPT").Value = "TP"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TEMP"
        ).Value = self.Temp
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES"
        ).Value = self.Pres

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("QCALC").Value)
        return q


class Turbine(Block):
    def __init__(self, name, press, inlet_stream):
        super().__init__(name, "COMPR")
        self.name = name
        self.press = press
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def expand(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "MODEL_TYPE"
        ).Value = "TURBINE"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TYPE"
        ).Value = "ISENTROPIC"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "OPT_SPEC"
        ).Value = "PRES"
        self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements("SEFF").Value = 0.85
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES"
        ).Value = self.press

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("WNET").Value)
        return q


class Compressor(Block):
    def __init__(self, name, press, inlet_stream):
        super().__init__(name, "COMPR")
        self.name = name
        self.press = press
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def compress(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "MODEL_TYPE"
        ).Value = "COMPRESSOR"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TYPE"
        ).Value = "ISENTROPIC"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "OPT_SPEC"
        ).Value = "PRES"
        self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements("SEFF").Value = 0.82
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES"
        ).Value = self.press

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("WNET").Value)
        return q

In [4]:
class Flowsheet:
    def __init__(self, sim, pure, max_iter, inlet_specs, Pressures):

        # Establish connection with ASPEN
        self.sim = sim

        # Characteristics of the environment
        self.d_actions = 10
        self.pure = pure
        self.max_iter = max_iter
        self.iter = 0
        self.actions_list = []
        self.co2 = 0.0
        self.Pressures = Pressures
        self.cooler_pdrop = 1e5  # 0.5e5
        self.heater_pdrop = 0  # 1e5
        self.hx_pdrop = 0.5e5  # 1e5
        self.value_step = "pre"
        self.splitter2ndstream = None

        # Declare the initial flowrate conditions
        self.inlet_specs = inlet_specs

        # Flowsheet
        self.info = {}

        self.turbine_count = 0
        self.cooler_count = 0
        self.compressor_count = 0
        self.heater_count = 0
        self.hex_count = 0
        self.mixer_count = 0
        self.splitter_count = 0

        self.reset()

    def get_outputs(self, sout):
        T = sout.get_temp()
        P = sout.get_press()
        Fco2 = sout.get_molar_flow("CO2")
        out_list = [T, P, Fco2]

        return out_list

    def step(self, action, sin):

        d_action = action[0]
        c_action = action[1]

        # ----------------------------------------- Turbine -----------------------------------------
        if d_action == 1:
            self.turbine_count += 1
            P_tur = c_action
            self.actions_list.append(f"T{self.turbine_count}")

            tur = Turbine(f"T{self.turbine_count}", P_tur, sin)
            sout = tur.expand()
            # self.sim.EngineRun()

            # if self.sim.Convergence():
            #     self.info[f"T{self.turbine_count}"] = [P_tur, self.get_outputs(sout)]

        # ----------------------------------------- Cooler -----------------------------------------
        elif d_action == 2:
            self.cooler_count += 1
            T_cooler = c_action
            P_cooler = self.Pressures[self.iter]
            self.actions_list.append(f"A{self.cooler_count}")

            cool = Cooler(f"A{self.cooler_count}", T_cooler, P_cooler, sin)

            sout = cool.cool()
            # self.sim.EngineRun()

            # if self.sim.Convergence():
            #     self.info[f"C{self.cooler_count}"] = [T_cooler, self.get_outputs(sout)]

        # ----------------------------------------- Compressor -----------------------------------------
        elif d_action == 3:
            self.compressor_count += 1
            P_comp = c_action
            self.actions_list.append(f"C{self.compressor_count}")

            comp = Compressor(f"C{self.compressor_count}", P_comp, sin)
            sout = comp.compress()
            # self.sim.EngineRun()

            # if self.sim.Convergence():
            #     self.info[f"C{self.compressor_count}"] = [
            #         P_comp,
            #         self.get_outputs(sout),
            #     ]

        # ----------------------------------------- Heater -----------------------------------------
        elif d_action == 4:
            self.heater_count += 1
            self.actions_list.append(f"H{self.heater_count}")
            T_heater = c_action
            P_heater = self.Pressures[self.iter]
            heater = Heater(f"H{self.heater_count}", T_heater, P_heater, sin)
            sout = heater.heat()
            # self.sim.EngineRun()

            # if self.sim.Convergence():
            #     self.info[f"H{self.heater_count}"] = [T_heater, self.get_outputs(sout)]

        # ----------------------------------------- Heat Exchanger -----------------------------------------
        elif d_action == 5:
            self.hex_count += 1
            self.actions_list.append(f"X{self.hex_count}")
            if self.hex_count == 1:
                DT_hex = c_action
                self.hex = HeatExchanger(f"X{self.hex_count}", DT_hex,sin)
                sout = self.hex.heat(self.hex_count, self.Pressures[self.iter])
            if self.hex_count == 2:
                self.hex.inlet_stream2 = sin
                sout = self.hex.heat(self.hex_count, self.Pressures[self.iter])
                
                # self.sim.EngineRun()

                # if self.sim.Convergence():
                #     self.info[f"HX{self.hex_count}"] = [DT_hex, self.get_outputs(sout)]

        # ----------------------------------------- Mixer -----------------------------------------
        elif d_action == 7:
            self.mixer_count += 1
            self.actions_list.append(f"M{self.mixer_count}")

            if self.mixer_count == 1:
                self.mixer = Mixer(f"M{self.mixer_count}", sin)
                sout = self.mixer.mix(self.mixer_count)
                sout = self.splitter2ndstream
            if self.mixer_count == 2:
                self.mixer.inlet_stream2 = sin
                sout = self.mixer.mix(self.mixer_count)

            # if self.sim.Convergence():
            #     self.info[f"M{self.mixer_count}"] = self.get_outputs(sout)
        # ----------------------------------------- Splitter -----------------------------------------
        elif d_action == 9:
            self.splitter_count += 1
            self.actions_list.append(f"S{self.splitter_count}")
            sr = c_action
            splitter = Splitter(f"S{self.splitter_count}", sr, sin)
            sout, self.splitter2ndstream = splitter.split()

            # self.sim.EngineRun()

            # if self.sim.Convergence():
            #     self.info[f"S{self.splitter_count}"] = self.get_outputs(sout)
        # ---------------------------------- Constraints and rewards ----------------------------------
        # if self.sim.Convergence():

        #     # Constraints

        #     # Cons 1: (Temperature inside of reactor no greater than 400°C)
        #     # if d_action in (4, 5) and sout.get_temp() <= 400:
        #     #     bonus_T = 0.4
        #     # else:
        #     #     bonus_T = 0.0

        #     # # Driving force (reduction of the amount of MeOH)
        #     # m_frac_prev = sin.get_molar_flow("METHANOL") / sin.get_total_molar_flow()
        #     # m_frac = sout.get_molar_flow("METHANOL") / sout.get_total_molar_flow()
        #     # bonus = m_frac_prev - m_frac

        #     # # Cons 2. Output purities
        #     # if not self.water_pure:
        #     #     w_frac = sout.get_molar_flow("WATER") / sout.get_total_molar_flow()
        #     #     self.water_pure = w_frac >= self.pure

        #     # if self.dme_out != 0:
        #     #     self.dme_pure = (
        #     #         self.dme_out.get_molar_flow("DME")
        #     #         / self.dme_out.get_total_molar_flow()
        #     #         >= self.pure
        #     #     )

        #     # penalty = 0
        #     # reward_flow = 0
        #     # dme_extra = 0

        #     # if self.iter >= self.max_iter:
        #     #     self.done = True

        #     #     if not self.water_pure or not self.dme_pure:
        #     #         penalty -= 15 * (self.pure - w_frac)
        #     # else:
        #     #     if self.water_pure and self.dme_pure:
        #     #         self.done = True
        #     #         reward_flow += 0.2 * (self.max_iter - self.iter)

        #     # if self.water_pure and not self.dme_pure:
        #     #     sout = self.dme_out

        #     # # Reward for more DME flow
        #     # if self.dme_pure and not self.dme_extra_added:
        #     #     dme_extra = self.dme_out.get_molar_flow("DME") / (self.Cao / 2)
        #     #     self.dme_extra_added = True  # Set the flag to True to indicate that dme_extra has been added

        #     # reward = cost + bonus + bonus_T + penalty + reward_flow + dme_extra
        #     reward = 0
        #     self.state = np.array(
        #         [
        #             sout.get_temp(),
        #             sout.get_press(),
        #             sout.get_mass_flow(),
        #             self.iter / self.max_iter,
        #         ]
        #     )

        # else:
        #     self.done = True
        #     reward = -8

        # Return step information
        self.iter += 1
        return self.state, self.done, self.info, sout
    
    def unit_report(self):
        units = [item.Name for item in self.sim.BLK.Elements]
        uo_dict = {}
        for unit in units:
            if unit.startswith("A") or unit.startswith("H"):
                uo_dict[unit] = self.sim.BLK.Elements(unit).Elements("Output").Elements("QCALC").Value
            elif unit.startswith("X"):
                uo_dict[unit] = self.sim.BLK.Elements(unit).Elements("Output").Elements("HX_DUTY").Value
            elif unit.startswith("T") or unit.startswith("C"):
                uo_dict[unit] = self.sim.BLK.Elements(unit).Elements("Output").Elements("WNET").Value

        total_turbine_power = sum(
            value for key, value in uo_dict.items() if key.startswith("T")
        )
        total_compressor_power = sum(
            value for key, value in uo_dict.items() if key.startswith("C")
        )
        net_power = total_turbine_power + total_compressor_power
        total_heating_duty = sum(
            value for key, value in uo_dict.items() if key.startswith("H")
        )
        total_cooling_duty = sum(
            value for key, value in uo_dict.items() if key.startswith("A")
        )
        net_heating_duty = total_heating_duty + total_cooling_duty
        return (uo_dict,net_power,net_heating_duty)
    
    def render(self):
        for i in self.info:
            print(f"{i}: {self.info[i]}")

    def reset(self):
        # Reset all instances
        self.iter = 0
        self.sim.Reinitialize()
        # inlet_specs is going to be a list of lists which contains variables and dictionaries for compounds
        self.state = np.zeros(
            (len(self.inlet_specs), len(self.inlet_specs[0])), dtype=np.float32
        )
        sin = [None] * len(self.inlet_specs)
        for i in range(len(self.inlet_specs)):
            T, P, compounds = self.inlet_specs[i]
            Fco2 = compounds["CO2"]
            sin[i] = Stream(f"N{i+1}", self.inlet_specs[i])

            self.state[i] = np.array([T, P, Fco2])

        self.info.clear()
        self.actions_list.clear()
        self.done = False

        self.turbine_count = 0
        self.cooler_count = 0
        self.compressor_count = 0
        self.heater_count = 0
        self.hex_count = 0
        self.mixer_count = 0
        self.splitter_count = 0

        return self.state, sin


In [5]:
classes = ["G", "T", "A", "C", "H", "a", "b", "1", "2", "-1", "-2", "E"]
def string_to_layout(sequence):
    """
    Converts a sequence to a layout tensor information to process in the optimization
    """
    one_hot_encoded = []
    i = 0
    while i < len(sequence):
        char = sequence[i]
        vector = [0] * len(classes)  # Initialize with zeros
        if char == "-":
            next_char = sequence[i + 1]
            unit = char + next_char
            if unit in classes:
                vector[classes.index(unit)] = 1
                i += 1  # Skip the next character since it forms a unit
        elif char in classes:
            vector[classes.index(char)] = 1
        one_hot_encoded.append(vector)
        i += 1
    return np.array(one_hot_encoded)

def bound_creation(layout):
    if layout[0][0]:
        layout = layout[1:]
    if layout[-1][-1]:
        layout = layout[:-1]
    units = layout
    # print(units)
    x = []
    splitter = False
    equipment = np.zeros(len(units)).tolist()
    bounds = list(range(len(units)))
    hx_token = 1
    for i in range(len(units)):
        unit_type = np.where(units[i] == 1)[0][0]
        if unit_type == 1:
            equipment[i] = 1
            bounds[i] = (74e5, 300e5)
        elif unit_type == 2:
            equipment[i] = 2
            bounds[i] = (32, 38)
            # TH
            # bounds[i] = (32.25, 530)
        elif unit_type == 3:
            equipment[i] = 3
            bounds[i] = (74e5, 300e5)
        elif unit_type == 4:
            equipment[i] = 4
            bounds[i] = (180, 530)
            # TH
            # bounds[i] = (32.25, 530)
        elif unit_type == 5:
            equipment[i] = 5
            if hx_token == 1:
                bounds[i] = (4, 11)
                # TH
                # bounds[i] = (3, 20)
            else:
                bounds[i] = (0, 0)
        elif unit_type == 7:
            equipment[i] = 7
            bounds[i] = (0, 0)
        elif unit_type == 9:
            equipment[i] = 9
            bounds[i] = (0.01, 0.99)
            # TH
            # bounds[i] = (0.1, 0.9)
            splitter = True
            branch_start = i
    if splitter == True:
        equipment = np.roll(equipment, -branch_start, axis=0).tolist()
        bounds = np.roll(bounds, -branch_start, axis=0).tolist()
    bounds.append((50, 160))
    # print(equipment)
    # print(bounds)
    return (equipment, bounds, x, splitter)

def decision_variable_placement(x, enumerated_equipment, equipment_length):
    approach_temp = 1
    split_ratio = 1
    hx_token = 1
    Temperatures = np.zeros(equipment_length)
    Pressures = np.zeros(equipment_length)
    mass_flow = np.ones(equipment_length) * x[-1]
    for index, equip in enumerated_equipment:
        if equip == 1:
            Pressures[index] = x[index]
        elif equip == 2:
            Temperatures[index] = x[index]
        elif equip == 3:
            Pressures[index] = x[index]
        elif equip == 4:
            Temperatures[index] = x[index]
        elif equip == 5 and hx_token == 1:
            approach_temp = x[index]
            hx_token = 0
        elif equip == 5 and hx_token == 0:
            x[index] = approach_temp
        elif equip == 9:
            split_ratio = x[index]
            branching_start = index
            branching_end1, branching_end2 = np.where(
                7 == np.array(enumerated_equipment)[:, 1]
            )[0]
            for i in range(branching_start, branching_end1):
                mass_flow[i] = mass_flow[i] * split_ratio
            for i in range(branching_end1, branching_end2):
                mass_flow[i] = mass_flow[i] * (1 - split_ratio)

    return (Pressures, Temperatures, approach_temp, split_ratio, mass_flow)

def Pressure_calculation(
    Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter=False
):
    cycle = 0
    if splitter == True:
        equipment = np.asarray(equipment)
        mixer1, mixer2 = np.where(7 == equipment)[0]

    while Pressures.prod() == 0:
        if (
            splitter == True
            and Pressures[mixer1 - 1] != 0
            and Pressures[mixer2 - 1] != 0
        ):
            Pressures[mixer2] = min(Pressures[mixer1 - 1], Pressures[mixer2 - 1])

        if Pressures.prod() == 0:
            for i in range(len(Pressures)):
                if Pressures[i] != 0:
                    if i == len(Pressures) - 1:
                        if equipment[0] == 2:
                            Pressures[0] = Pressures[i] - cooler_pdrop
                        if equipment[0] == 4:
                            Pressures[0] = Pressures[i] - heater_pdrop
                        if equipment[0] == 5:
                            Pressures[0] = Pressures[i] - hx_pdrop
                        if equipment[0] == 9:
                            Pressures[0] = Pressures[i]
                            Pressures[mixer1] = Pressures[i]

                    else:
                        if equipment[i + 1] == 2:
                            Pressures[i + 1] = Pressures[i] - cooler_pdrop
                        if equipment[i + 1] == 4:
                            Pressures[i + 1] = Pressures[i] - heater_pdrop
                        if equipment[i + 1] == 5:
                            Pressures[i + 1] = Pressures[i] - hx_pdrop
                        if equipment[i + 1] == 9:
                            Pressures[i + 1] = Pressures[i]
                            Pressures[mixer1] = Pressures[i]
            cycle += 1
        if cycle == 5:
            break

    return Pressures
cooler_pdrop = 1e5  # 0.5e5
heater_pdrop = 0  # 1e5
hx_pdrop = 0.5e5  # 1e5

global sim
cwd = os.getcwd() # get current working directory
os.chdir(cwd) # change to the directory where the script is located
sim = Simulation("sco2.bkp",cwd,True) # create the base case of the simulation


Working Directory: c:\Users\m85830ak\Python\flowsheet-desing-main\SCO2
Aspen File: sco2.bkp


In [34]:
layouts = np.load("M2_data_F8_layouts.npy", allow_pickle=True)
positions = np.load("M2_data_F8_positions.npy", allow_pickle=True)
max_number = len(layouts)
i = 3500
# log = []
starting_layout = i # change this to start from a different layout
layouts = layouts[starting_layout:]
positions = positions[starting_layout:]
for text,position in zip(layouts, positions):
    layout = string_to_layout(text)
    equipment, _, _, splitter = bound_creation(layout)
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    (Pressures, Temperatures, approach_temp, split_ratio, mass_flow
    ) = decision_variable_placement(position, enumerated_equipment, equipment_length)
    Pressures = Pressure_calculation(
        Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter
    )
    #find the last non-zero temperature
    last_temp_index = np.where(Temperatures != 0)[0][-1]
    position_without_m = position[:-1]  
    if last_temp_index != equipment_length - 1:
        # roll the arrays until the last temperature is at the end
        Pressures = np.roll(Pressures, -last_temp_index - 1)
        Temperatures = np.roll(Temperatures, -last_temp_index - 1)
        mass_flow = np.roll(mass_flow, -last_temp_index - 1)
        equipment = np.roll(equipment, -last_temp_index - 1)
        position_without_m = np.roll(position_without_m, -last_temp_index - 1)
    
    env_kwargs = {
    "sim": sim,
    "pure": 0.99, 
    "max_iter": 25, 
    "inlet_specs": [[Temperatures[-1].item(), Pressures[-1].item(), {"CO2": mass_flow[-1].item()}],],
    "Pressures": Pressures.tolist()
}
    try:
        attempts = 0          
        while attempts<3:
            try:
                state = None
                trials = 0
                while state is None:
                    env = Flowsheet(**env_kwargs) # create a flowsheet instance
                    state,inlet_streams = env.reset()
                    sin = inlet_streams[0]
                    trials += 1
                    
                for equip, value in zip(equipment, position_without_m):
                    action = [equip, value]
                    if   action[0] == 1:
                        state,done,info, sin = env.step(action, sin)
                    elif action[0] == 2:
                        state,done,info, sin = env.step(action, sin)
                    elif action[0] == 3:
                        state,done,info, sin = env.step(action, sin)
                    elif action[0] == 4:
                        state,done,info, sin = env.step(action, sin)
                    elif action[0] == 5:
                        state_,done,info, sin = env.step(action, sin)
                    elif action[0] == 7:
                        state,done,info,sin = env.step(action, sin)
                    elif action[0] == 9:
                        state,done,info, sin = env.step(action, sin)
            except Exception as e:
                print(f"Error in processing layout 1stexcp {i}: {e}")
                print(i,equipment)
                attempts += 1
                env.sim.Reinitialize()
                print(attempts)
                continue
            else:
                break
        env.sim.EngineRun()
        if not env.sim.Convergence():
            env.hex.switch_streams()
            env.sim.EngineReinit()
            env.sim.EngineRun()
        if len([item.Name for item in sim.BLK.Elements]) and env.sim.Convergence():
            uo_dict, net_power, net_heating_duty = env.unit_report()
            log.append([i, text.item(), position, 1, uo_dict, net_power, net_heating_duty])
        else:
            log.append([i, text, position, 0,0,0,0])
    except Exception as e:
        print(f"Error processing layout 2ndexcp {i}: {e}")
        log.append([i, text, position, -1,0,0,0])
    i += 1
    if i >= max_number:
        break
    if i % 100 == 0:
        #metrics
        print(f"Processed {i} layouts out of {max_number}.")
        

Error in processing layout 1stexcp 3552: (-2147352567, 'Exception occurred.', (6, 'VAgetlen (see errcode)', 'AE_NOINSTANCE', None, 0, 0), None)
3552 [3 5 9 4 1 7 1 7 4 1 5 2]
1
Processed 3600 layouts out of 4267.
Processed 3700 layouts out of 4267.
Error in processing layout 1stexcp 3780: 'NoneType' object has no attribute 'name'
3780 [7 1 5 9 2 3 1 7 2 3 5 4]
1
Error in processing layout 1stexcp 3780: 'NoneType' object has no attribute 'name'
3780 [7 1 5 9 2 3 1 7 2 3 5 4]
2
Error in processing layout 1stexcp 3780: 'NoneType' object has no attribute 'name'
3780 [7 1 5 9 2 3 1 7 2 3 5 4]
3
Processed 3800 layouts out of 4267.
Error in processing layout 1stexcp 3854: (-2147352567, 'Exception occurred.', (6, 'VAgetlen (see errcode)', 'AE_NOINSTANCE', None, 0, 0), None)
3854 [3 9 4 1 7 2 5 4 1 7 1 5 2]
1
Processed 3900 layouts out of 4267.
Error in processing layout 1stexcp 3916: (-2147352567, 'Exception occurred.', (6, 'VAgetlen (see errcode)', 'AE_NOINSTANCE', None, 0, 0), None)
3916 [3 

In [36]:
log_df = pd.DataFrame(log, columns=["Index", "Layout", "Position", "Status", "Unit Outputs", "Net Power", "Net Heating Duty"])
log_df.to_csv("simulation_log_withRHEX.csv", index=False)
print(f"Number of successful simulations: {len(log_df[log_df['Status'] == 1])}")
print(f"Number of failed simulations: {len(log_df[log_df['Status'] == 0])}")
print(f"Number of errored simulations: {len(log_df[log_df['Status'] == -1])}")
# print(log_df) # print the first 5 rows of the log dataframe

Number of successful simulations: 3370
Number of failed simulations: 880
Number of errored simulations: 20


In [37]:
env.sim.AspenSimulation.Quit()  # Close the Aspen simulation

In [7]:
print([item.Name for item in sim.BLK.Elements]) # get the list of unit operations
print([item.Name for item in sim.STRM.Elements]) # get the list of streams
print(dir(env.sim.AspenSimulation)) # get the list of methods and attributes of the AspenSimulation class

['A1', 'C1', 'C2', 'H1', 'H2', 'T1', 'X1']
['A1OUT', 'C1OUT', 'C2OUT', 'H1OUT', 'H2OUT', 'N1', 'T1OUT', 'X1OUT1', 'X1OUT2']
['Activate', 'AdviseParent', 'Application', 'AutoSave', 'CLSID', 'Choose', 'Close', 'CloseDocument', 'CreateRouteTree', 'DeleteSelection', 'EditCompoundDocument', 'Engine', 'EngineServer', 'EngineSimulation', 'Export', 'FailedToOpenDescription', 'FailedToOpenKey', 'FullName', 'Generate', 'GetNew', 'GetNew3', 'Import', 'InitFromArchive', 'InitFromArchive2', 'InitFromArchive3', 'InitFromFile', 'InitFromFile2', 'InitFromTemplate', 'InitFromTemplate2', 'InitFromXML', 'InitNew', 'InitNew2', 'LibRef', 'LoadLink', 'Name', 'New', 'New2', 'New3', 'NewAsync', 'NewSelection', 'Parent', 'ProcessId', 'Quit', 'Readback', 'Reconcile', 'RefreshOff', 'Reinit', 'Restore', 'Restore2', 'RootModel', 'Run', 'Run2', 'RunScript', 'Save', 'Save2', 'SaveAs', 'SaveAs2', 'SaveLink', 'SaveSelection', 'Selection', 'SetCompat', 'SetParent', 'StrongReferences', 'SuppressDialogs', 'Tree', 'UIDisa

In [18]:
attempts = 0
while attempts<3:
    try:
        env = Flowsheet(**env_kwargs) # create a flowsheet instance
        state,inlet_streams = env.reset()
        sin = inlet_streams[0]
        for equip, value in zip(equipment, position_without_m):
            action = [equip, value]
            if   action[0] == 1:
                state,done,info, sin = env.step(action, sin)
            elif action[0] == 2:
                state,done,info, sin = env.step(action, sin)
            elif action[0] == 3:
                state,done,info, sin = env.step(action, sin)
            elif action[0] == 4:
                state,done,info, sin = env.step(action, sin)
            elif action[0] == 5:
                state_,done,info, sin = env.step(action, sin)
            elif action[0] == 7:
                state,done,info,sin = env.step(action, sin)
            elif action[0] == 9:
                state,done,info, sin = env.step(action, sin)
    except:
        attempts += 1
        env.sim.Reinitialize()
        print(f"Error encountered, retrying... Attempt {attempts}/3")
        continue
    else:
        print(attempts)
        print("Simulation completed successfully.")
        break
env.sim.EngineRun()
if not env.sim.Convergence():
    # env.hex.switch_streams()
    env.sim.EngineReinit()
    env.sim.EngineRun()
if env.sim.Convergence():
    print("Simulation converged successfully.")

com_error: (-2147417848, 'The object invoked has disconnected from its clients.', None, None)

In [ ]:
for equip,T,P in zip(equipment, Temperatures, Pressures):
    print(f"Equipment: {equip}, Temperature: {T}, Pressure: {P}")

In [ ]:
# PRES_COLD, PRES_HOT pressure dropped outlet for heat exchanger
# DELT-COLD 

In [ ]:
sim.CloseAspen()  # Close the Aspen simulation

In [ ]:
# classes = ["G", "T", "A", "C", "H", "a", "b", "1", "2", "-1", "-2", "E"]
# def string_to_equipment(sequences, classes):
#     """
#     Converts a list of sequences to a list of lists of equipments
#     """
#     char_to_int = dict((c, i) for i, c in enumerate(classes))
#     equipments = []
#     for sequence in sequences:
#         equipment = []
#         splitter = False
#         for char in sequence:
#             try:
#                 equipment.append(char_to_int[char])
#             except:
#                 equipment.append(char_to_int["-1"])
#                 splitter = True
#         if splitter == True:
#             equipment.pop(equipment.index(char_to_int["-1"]) + 1)
#             splitter = False
#         equipments.append(equipment)
#     return equipments
# equipments = string_to_equipment(layouts, classes)
# for equipment in equipments:
#     if equipment[0] == 0:
#         equipment.pop(0)
#     if equipment[-1] == 11:
#         equipment.pop(-1)
        
# equipments = np.array(equipments, dtype=object)
# positions = np.array(positions, dtype=object)
# # roll the equipments and positions until the last element of each equipment 4
# heater_index = np.zeros(len(equipments), dtype=int)
# i = 0
# for equipment in equipments:
#     heater_index[i] = equipment.index(4) 
#     i += 1
# mass_flowrates = [positions[i][-1] for i in range(len(positions))]
# positions = [pos[:-1] for pos in positions]
# positions = np.array(positions, dtype=object)
# for i in range(len(equipments)):
#     equipments[i] = np.roll(equipments[i], -heater_index[i]-1)
#     positions[i] = np.roll(positions[i], -heater_index[i]-1)
# # for i in range(len(equipments)):
# last_pressure_equipment = np.zeros(len(equipments), dtype=int)
# for i in range(len(equipments)):
#     turbine_index = np.where(equipments[i] == 1)[0].max()
#     compressor_index = np.where(equipments[i] == 3)[0].max()
#     if compressor_index < turbine_index:
#         last_pressure_equipment[i] = turbine_index
#     else:
#         last_pressure_equipment[i] = compressor_index

# positions = [np.append(positions[i], mass_flowrates[i]) for i in range(len(positions))]


# print(heater_index[0],equipments[0], positions[0])

# print("\nNumber of layouts:", len(equipments),len(positions))
# # Preparing the inlet conditions
# def prepare_inlet_conditions(equipments,positions):
#     inlet_conditions = np.zeros((len(equipments), 3))
#     print(inlet_conditions.shape)
#     #Mass flow rate
#     inlet_conditions[:, 0] = [positions[i][-1] for i in range(len(positions))]
#     #Temperature
#     inlet_conditions[:, 1] = [positions[i][-2] for i in range(len(positions))]
#     #Pressure
#     pressure = positions
# # prepare_inlet_conditions(equipments, positions)



In [ ]:
# import os
# import win32com.client as win32
# from scoenv import Flowsheet
# from scoSimulation import *

# cwd = os.getcwd() # get current working directory
# os.chdir(cwd) # change to the directory where the script is located
# sim = Simulation("base.bkp",cwd,True) # create the base case of the simulation
# env_kwargs = {
#         "sim": sim,
#         "pure": 0.99, 
#         "max_iter": 15, 
#         "inlet_specs": [[298, 2.4, {"EO": 0.0, "W": 26.31, "EG": 0.0, "DEG": 0.0}],[298, 2.4, {"EO": 27.62, "W": 0.0, "EG": 0.0, "DEG": 0.0}],
#                         [360.27,2.4,{"EO": 0.86483855131462, "W": 521.25960426966, "EG": 0.253500051247613, "DEG": 1.05869515878343E-07}],
#         ]
# }

# env = Flowsheet(**env_kwargs) # create a flowsheet instance

In [ ]:
env.sim.CloseAspen()

In [ ]:
# #without recycle
# observations = []
# state,sin = env.reset() # reset the environment to its initial state
# observations.append(state) # append the initial state to the observations list
# action =[0,0]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [1,355]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [4,3.75]
# for i in range(9):
#     state,reward,done,info,sin = env.step(action,sin)
#     observations.append(state)
# action = [3,398]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [6, 0.71] # without recycle
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [2,2.4]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)


In [ ]:
# #with recycle
# env_kwargs = {
#         "sim": sim,
#         "pure": 0.99, 
#         "max_iter": 15, 
#         "inlet_specs": [[298, 2.4, {"EO": 0.0, "W": 26.31, "EG": 0.0, "DEG": 0.0}],[298, 2.4, {"EO": 27.62, "W": 0.0, "EG": 0.0, "DEG": 0.0}],
#         ]
# }

# env = Flowsheet(**env_kwargs) # create a flowsheet instance
# observations = []
# state,sin = env.reset() # reset the environment to its initial state
# observations.append(state) # append the initial state to the observations list
# action =[0,0]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [1,355]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [4,3.75]
# for i in range(9):
#     state,reward,done,info,sin = env.step(action,sin)
#     observations.append(state)
# action = [3,398]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [7, 0.71] # with recycle
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)

In [ ]:
# observations = []
# state,sin = env.reset() # reset the environment to its initial state
# observations.append(state) # append the initial state to the observations list
# action =[0,0]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [1,355]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [5,8]
# state,reward,done,info,sin = env.step(action,sin)
# action = [3,398]
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)
# action = [7, 0.71] # with recycle
# state,reward,done,info,sin = env.step(action,sin)
# observations.append(state)


In [ ]:
# env.sim.EngineReinit() # reinitialize the simulation engine
# env.sim.EngineRun() # run the simulation engine

In [ ]:
# #print observations in a readable format
# with np.printoptions(precision=2, suppress=True):
#     print("Observations:")
#     for i, obs in enumerate(observations):
#         print(f"Step {i}: {obs}")
#     print("Final reward:", reward)
#     print("Final done:", done)
#     print("Final info:", info)
    

In [ ]:
# sim.CloseAspen() # close the Aspen simulation